# Basic Usage

## Imports

In [1]:
from autora.theorist.rnn_sindy_rl import RNNSindy
from autora.state import StandardState, on_state, Delta
from autora.variable import VariableCollection, Variable

import numpy as np
import pandas as pd

## Constants

In [2]:
TRIALS_PER_PARTICIPANT = 30

## Set up variables
independent variable is "reward-trajectory": A 2 x n_trials Vector with entries between 0 and 1
dependent variable is "choice-trajectory": A 2 x n_trials Vector with boolean entries (one hot encoded)

In [3]:
variables = VariableCollection(
    independent_variables=[Variable(name="reward-trajectory")],
    dependent_variables=[Variable(name="choice-trajectory")]
)

# State
We use the StandardState

In [4]:
state = StandardState(variables=variables)

## Components
### Experimentalist
Our experimentalist creates n random reward-trajectories

In [5]:
@on_state()
def experimentalist_random_trajectory_on_state(num_samples, n_trials=TRIALS_PER_PARTICIPANT):
    """
    This is creates `num_samples` randomized reward trajectories of length `n_trials`
    """
    trajectories = np.random.randint(0, 2, (num_samples, n_trials, 2))
    trajectories_list = [trajectory for trajectory in trajectories]
    conditions = pd.DataFrame({'reward-trajectory': trajectories_list})

    return Delta(conditions=conditions)

For example, let's create 3 conditions:

In [6]:
state = experimentalist_random_trajectory_on_state(state, num_samples=3)
state.conditions

,reward-trajectory
0,"[[1, 1], [1, 1], [1, 1], [0, 0], [0, 0], [0, 0..."
1,"[[1, 0], [0, 1], [0, 0], [1, 1], [1, 1], [1, 0..."
2,"[[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1..."


## Experiment Runner
Here, we create a rando choice trajectory

In [7]:
@on_state()
def runner_on_state(conditions):
    def generate_choice():
        return [0, 1] if np.random.rand() > 0.5 else [1, 0]
    trajectories_list = [
        np.array([generate_choice() for _ in range(len(reward_trajectory))])
        for reward_trajectory in conditions['reward-trajectory']
    ]
    experiment_data = pd.DataFrame({'reward-trajectory': list(conditions['reward-trajectory']),
                                    'choice-trajectory': trajectories_list}, dtype=object)
    return Delta(experiment_data=experiment_data)

In [8]:
state = runner_on_state(state)
state.experiment_data

,reward-trajectory,choice-trajectory
0,"[[1, 1], [1, 1], [1, 1], [0, 0], [0, 0], [0, 0...","[[1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0..."
1,"[[1, 0], [0, 1], [0, 0], [1, 1], [1, 1], [1, 0...","[[0, 1], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0..."
2,"[[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1...","[[0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0..."


## Theorist
We use the ExampleRegressor to fit:

@on_state()
def theorist_on_state(experiment_data):
    x, y = _get_x_y(experiment_data)

    return Delta(models=[theorist.fit(x, y)])


In [9]:
regressor = RNNSindy(2)
@on_state()
def theorist_on_state(experiment_data):

    x = experiment_data['reward-trajectory']
    y = experiment_data['choice-trajectory']

    return Delta(models=[regressor.fit(x, y)])

In [11]:
state = theorist_on_state(state)
state.models[-1]

Training the RNN...
Epoch 1/100 --- Loss: 0.4490971; Time: 0.1421s; Convergence value: 5.51e-01
Epoch 2/100 --- Loss: 0.2854415; Time: 0.1356s; Convergence value: 3.55e-01
Epoch 3/100 --- Loss: 0.4515755; Time: 0.1584s; Convergence value: 2.90e-01
Epoch 4/100 --- Loss: 0.3536690; Time: 0.1438s; Convergence value: 2.40e-01
Epoch 5/100 --- Loss: 0.3526500; Time: 0.1400s; Convergence value: 1.89e-01
Epoch 6/100 --- Loss: 0.4983991; Time: 0.1324s; Convergence value: 1.81e-01
Epoch 7/100 --- Loss: 0.4088033; Time: 0.1293s; Convergence value: 1.67e-01
Epoch 8/100 --- Loss: 0.4070661; Time: 0.1389s; Convergence value: 1.44e-01
Epoch 9/100 --- Loss: 0.4171735; Time: 0.1271s; Convergence value: 1.27e-01
Epoch 10/100 --- Loss: 0.3878347; Time: 0.1261s; Convergence value: 1.15e-01
Epoch 11/100 --- Loss: 0.3339262; Time: 0.1288s; Convergence value: 1.09e-01
Epoch 12/100 --- Loss: 0.3889047; Time: 0.1326s; Convergence value: 1.03e-01
Epoch 13/100 --- Loss: 0.3775082; Time: 0.1351s; Convergence valu

RNNSindy(n_actions=2)